## Extracción de Datos de Manera Estructura

In [ ]:
import boto3
import json

#### Vamos a extraer atributos de Anuncios de Propiedades

In [ ]:
prompt="""
¡OPORTUNIDAD ÚNICA! - La Casa más Peculiar de Silicon Valley

Ponemos a la venta esta extraordinaria propiedad ubicada en 404 Error Street, Palo Alto, Silicon Valley. Esta casa de 180m² construida en 1999 fue anteriormente propiedad de un excéntrico programador que la convirtió en el sueño de cualquier geek.

Cuenta con 3 dormitorios y 2 baños, pero lo más destacable son sus características únicas: todas las luces se controlan por comandos de voz en lenguaje Python, el frigorífico envía tweets cuando se está quedando sin comida, y el jardín tiene un sistema de riego controlado por IA que ha desarrollado su propia personalidad (a veces se niega a regar las plantas que no le caen bien).

La casa incluye una sala de servidores convertida en spa, donde el calor de los ordenadores mantiene la temperatura perfecta para el jacuzzi. El garaje tiene capacidad para 2 coches o 200 placas Arduino, según preferencias.

Estado: Bueno (excepto por algunos bugs menores en el sistema domótico)
Precio: $2,500,000 (se acepta pago en Bitcoin)

Características adicionales:
- WiFi que alcanza hasta la Luna
- Sistema de seguridad que identifica a los visitantes por su historial de GitHub
- Tostadora que imprime memes en el pan
- Puerta principal que solo se abre si resuelves un acertijo de programación

¡Una verdadera joya para los amantes de la tecnología!
"""

#### Definimos esquema con los mayores detalles posibles, para que el LLM entienda

In [ ]:
tool_config = {
    "tools": [{
        "toolSpec": {
            "name": "extract_property",
            "description": "Extract property listing information",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "property": {
                            "type": "object",
                            "properties": {
                                "title": {
                                    "type": "string",
                                    "description": "Title of the property listing"
                                },
                                "price": {
                                    "type": "number",
                                    "description": "Price in local currency"
                                },
                                "location": {
                                    "type": "object",
                                    "properties": {
                                        "street": {"type": "string"},
                                        "neighborhood": {"type": "string"},
                                        "city": {"type": "string"}
                                    }
                                },
                                "size": {
                                    "type": "number",
                                    "description": "Size in square meters"
                                },
                                "rooms": {
                                    "type": "object",
                                    "properties": {
                                        "bedrooms": {"type": "integer"},
                                        "bathrooms": {"type": "integer"}
                                    }
                                },
                                "features": {
                                    "type": "array",
                                    "items": {
                                        "type": "string",
                                        "description": "Special features or amenities"
                                    }
                                },
                                "condition": {
                                    "type": "string",
                                    "enum": ["new", "good", "needs_renovation", "under_construction"]
                                },
                                "year_built": {
                                    "type": "integer",
                                    "description": "Year the property was built"
                                }
                            }
                        }
                    }
                }
            }
        }
    }]
}

In [ ]:
# Iniciamos cliente
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')

In [ ]:
# Invocamos LLM mediante Bedrock
model_response = bedrock_client.converse(
    modelId="us.amazon.nova-lite-v1:0",
   system=[{"text": "You are an expert extractor that compiles details, from real estate properties"}],
    messages=[{"role": "user", "content": [{"text": prompt}]}],
    inferenceConfig={"temperature": 0},
    toolConfig=tool_config
)

In [ ]:
# Show response with JSON structure
formatted_response = json.dumps(model_response['output']['message'], indent=2, ensure_ascii=False)
print(formatted_response)

In [ ]:
# Show response with JSON structure
formatted_response = json.dumps(model_response['output']['message']['content'][1]['toolUse']['input'], indent=2, ensure_ascii=False)
print(formatted_response)